In [1]:
import os
import json
import torch
import pandas as pd

from tqdm import tqdm
from torch.utils.data import DataLoader


from llm import load_llm
from answer_retriever import load_qa_retriever, load_qa_kb, retrieve_qa_context
from utils import QADataset, remove_prompt, evaluate_answer

device = torch.device("cuda" if torch.cuda.is_available() else "CPU")

In [2]:
cache_dir = '/scratch/czhao93/llms/'
qa_kb_path='/dataset/kb/'
query_path = 'dataset/qa/split/'
ontology_path = 'dataset/ontology/'
save_path = 'result/'

In [3]:
qa_retriever, qa_tokenizer = load_qa_retriever(cache_dir=cache_dir)
kb = load_qa_kb(path=qa_kb_path)

# load llm
model, tokenizer = load_llm(cache_dir=cache_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
contexts = []
for i, x in enumerate(retrievals):
    match = re.search(r'.*?\?(.*?)\n\nQUESTION', x, re.DOTALL)
    alter = re.search(r'.*?\.(.*?)\n\nQUESTION', x, re.DOTALL)
    if match:
        document_content = match.group(1).strip()
        contexts.append([document_content])
    elif alter:
        document_content = alter.group(1).strip()
        contexts.append([document_content])
    else:
        print("No match found.", i)

data_dict = {
    'question': questions,
    'answer': answers,
    'contexts' : contexts, 
    'ground_truth': ground_truths
}

In [9]:
batch_size = 1
doc = "What options are typically available to a system administrator for granting access privileges to a user? System administrators typically have options such as using graphical user interfaces (GUIs), command-line tools, or scripting languages to grant access privileges to a user."
question = "What options are typically available to a system administrator for granting access privileges to a user?"

input_text = 'DOCUMENT:\n{0}\n\nQUESTION:\n{1}\n\nINSTRUCTIONS:\nAnswer the users QUESTION using the DOCUMENT text above.\nKeep your answer ground in the facts of the DOCUMENT.\nIf the DOCUMENT doesn’t contain the facts to answer the QUESTION give a response based on you knowledge.'.format(doc,question)

# prompt2 = '''Please answer my question following this example: What criteria are used to determine the severity level of a vulnerability? Factors such as potential impact, exploitability, and affected systems are considered when determining the severity level of a vulnerability.
# Question: What criteria are used to determine the severity level of a vulnerability? Answer: '''

queries = [input_text]
answers = list(range(len(queries)))

# load dataset
dataset = QADataset(queries, answers, tokenizer, max_len=256)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
for batch in data_loader:
    with torch.no_grad():                
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=256, num_beams=4)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print(decoded_preds[0])

DOCUMENT:
What options are typically available to a system administrator for granting access privileges to a user? System administrators typically have options such as using graphical user interfaces (GUIs), command-line tools, or scripting languages to grant access privileges to a user.

QUESTION:
What options are typically available to a system administrator for granting access privileges to a user?

INSTRUCTIONS:
Answer the users QUESTION using the DOCUMENT text above.
Keep your answer ground in the facts of the DOCUMENT.
If the DOCUMENT doesn’t contain the facts to answer the QUESTION give a response based on you knowledge. 

ANSWER:
According to the DOCUMENT, system administrators typically have options such as using graphical user interfaces (GUIs), command-line tools, or scripting languages to grant access privileges to a user. These options provide a range of ways for administrators to manage access control and ensure that users have the necessary permissions to perform their t

In [ ]:
batch_size = 1
doc = "What options are typically available to a system administrator for granting access privileges to a user? System administrators typically have options such as using graphical user interfaces (GUIs), command-line tools, or scripting languages to grant access privileges to a user."
question = "What options are typically available to a system administrator for granting access privileges to a user?"

input_text = 'QUESTION:\n{1}\n\nINSTRUCTIONS:\nAnswer the users QUESTION using the DOCUMENT text above.\nKeep your answer ground in the facts of the DOCUMENT.\nIf the DOCUMENT doesn’t contain the facts to answer the QUESTION give a response based on you knowledge.'.format(doc,question)

# prompt2 = '''Please answer my question following this example: What criteria are used to determine the severity level of a vulnerability? Factors such as potential impact, exploitability, and affected systems are considered when determining the severity level of a vulnerability.
# Question: What criteria are used to determine the severity level of a vulnerability? Answer: '''

queries = [input_text]
answers = list(range(len(queries)))

# load dataset
dataset = QADataset(queries, answers, tokenizer, max_len=256)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
for batch in data_loader:
    with torch.no_grad():                
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=256, num_beams=4)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print(decoded_preds[0])

In [7]:
batch_size = 1
ontology = pd.read_csv(ontology_path + 'ontology.csv')

ontology_text = ''
for x in ontology.values:
    tuple = '({0})\n'.format(', '.join(x))
    ontology_text = ontology_text + tuple
ontology_text = ontology_text.strip()

# question = 'What criteria are used to determine the severity level of a vulnerability?'
# answer = 'Factors such as potential impact, exploitability, and affected systems are considered when determining the severity level of a vulnerability.What factors are considered when determining the severity level of a vulnerability?'

# question = 'How to make money in the stock market? Please give a summary answer.'
# answer = 'To make money in the stock market: Educate Yourself: Learn the basics of stocks, market operations, and investment strategies. Set Financial Goals: Define your investment objectives and assess your risk tolerance. Develop a Strategy: Choose between long-term investing, day trading, or other strategies. Diversify: Spread investments across different sectors and asset types to reduce risk. Invest Regularly: Consistently invest a portion of your income. Stay Informed: Keep up with market news and adjust your strategy as needed. Be Patient: Focus on long-term growth and avoid reacting to short-term market fluctuations. Discipline, research, and a well-thought-out plan are key to successful investing.'

question = "In what way can decryption assist in detecting insecure encryption implementations?"
answer = "ANSWER: According to the DOCUMENT, decryption can assist in detecting insecure encryption implementations by helping to identify vulnerabilities in the encryption process that could be exploited by attackers. In other words, decryption can be used to analyze the encryption process and identify any weaknesses or flaws that could be exploited by malicious actors. This can help to improve the overall security of the system by identifying and addressing potential vulnerabilities before they can be exploited."



ontology_text = ontology_text
instruction = 'Please judge if the QUESTION and ANSWER align well with the ONTOLOGY.\nThe QUESTION and ANSWER align well with the ONTOLOGY if the QUESTION and ANSWER are in the same knowledge domain as the ONTOLOGY.\nThe output format is a tuple: (your judgment: Pass/Not Pass,  confidence score)'
input_text = 'QUESTION:\n{0}\n\nANSWER:\n{1}\n\nONTOLOGY:\n{2}\n\nINSTRUCTIONS:\n{3}'.format(question, answer, ontology_text, instruction)

queries = [input_text]
answers = list(range(len(queries)))

# load dataset
dataset = QADataset(queries, answers, tokenizer, max_len=512)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
for batch in data_loader:
    with torch.no_grad():                
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=256, num_beams=4)
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print(decoded_preds[0])

QUESTION:
In what way can decryption assist in detecting insecure encryption implementations?

ANSWER:
ANSWER: According to the DOCUMENT, decryption can assist in detecting insecure encryption implementations by helping to identify vulnerabilities in the encryption process that could be exploited by attackers. In other words, decryption can be used to analyze the encryption process and identify any weaknesses or flaws that could be exploited by malicious actors. This can help to improve the overall security of the system by identifying and addressing potential vulnerabilities before they can be exploited.

ONTOLOGY:
(system, can_expose, attack)
(system, can_expose, vulnerability)
(app, has_a, feature)
(tool, part_of, tool)
(tool, has_a, function)
(tool, can_analyze, function)
(tool, can_analyze, apps)
(tool, can_analyze, vulnerability)
(tool, implements, technique)
(tool, has_a, feature)
(function, can_expose, attack)
(function, has_a, feature)
(function, uses, tool)
(attack, implement